<a href="https://colab.research.google.com/github/xychong/edgeaimonitoring/blob/main/Preprocessing/Preprocessing%5BAmbience%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import librosa, librosa.display
import numpy as np
import os

In [ ]:
pathname = "/content/drive/MyDrive/FYP Data/Ambience"

In [ ]:
# Display all filenames
sum = 0
for filename in os.listdir(pathname):
  sum += 1
  print(filename)

537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav
531947__straget__the-rain-falls-against-the-parasol.wav
527664__straget__thunder.wav
401060__naturenotesuk__weather-rain.wav
364663__jmiddlesworth__bird-call-in-spring.wav
freesound_ambience_0012.wav
own_ambience_0003.wav
own_ambience_0002.wav
own_ambience_0001.wav
soundsnap_ambience_0002.wav
own_ambience_0004.wav
soundsnap_ambience_0006.wav
soundsnap_ambience_0016.wav
soundsnap_ambience_0021.wav
soundsnap_ambience_0035.wav
soundsnap_ambience_0038.wav


In [ ]:
# Check that all files are included
print (sum)

16


In [ ]:
n_fft = 2048
hop_length = 512
for filename in os.listdir(pathname):
  x,sr = librosa.load(pathname + "/" + filename, sr = 44100) # load audio file with sampling rate 44.1k Hz
  stft = librosa.stft(x, n_fft = n_fft, hop_length = hop_length)
  amplitudeindB = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
  print (np.max(abs(amplitudeindB))) # gives us max dB value of audio for each audio file

80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0


This means that maximum audio level is at 80dB for all audio files in "Ambience" dataset.

In [ ]:
def displayTime(startFrame, endFrame):    
    print(' start time: ' + str(startFrame/sr) + ', end time: ' + str(endFrame/sr))

In [ ]:
# Obtain non-silent intervals within each track
# Allows me to have a better idea of how to trim each track into the desired 4s duration
count = 0
for filename in os.listdir(pathname):
  x,sr = librosa.load(pathname + "/" + filename, sr = 44100)
  nonmuteportions = librosa.effects.split(x) # default top_db = 60 dB
  for i in nonmuteportions: 
    print(filename)   
    displayTime(i[0],i[1])

537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav
 start time: 0.0, end time: 81.42875283446712
531947__straget__the-rain-falls-against-the-parasol.wav
 start time: 0.0, end time: 60.15170068027211
527664__straget__thunder.wav
 start time: 0.0, end time: 33.625079365079365
401060__naturenotesuk__weather-rain.wav
 start time: 0.0, end time: 301.3950113378685
364663__jmiddlesworth__bird-call-in-spring.wav
 start time: 0.0, end time: 131.5526530612245
freesound_ambience_0012.wav
 start time: 0.0, end time: 70.20934240362811
own_ambience_0003.wav
 start time: 0.0, end time: 60.546031746031744
own_ambience_0002.wav
 start time: 0.034829931972789115, end time: 60.65
own_ambience_0001.wav
 start time: 0.034829931972789115, end time: 60.46333333333333
soundsnap_ambience_0002.wav
 start time: 0.0, end time: 16.21469387755102
own_ambience_0004.wav
 start time: 0.0, end time: 30.023401360544216
soundsnap_ambience_0006.wav
 start time: 0.0, end time: 43.56267573696145
sound

All the audio files are >4s in duration.

From above data, there are not many instances of silences detected wihtin the "Ambience" dataset.

We can just jump straight into splitting each audio track into multiple segments of 4s each.

In [ ]:
from pydub import AudioSegment
from pydub.utils import make_chunks

In [ ]:
# Splitting audio tracks >4s into chunks of 4s each
# The final chunk may be <4s depending on total audio duration
count = 0
chunk_length = 4000
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  if audio.duration_seconds > 4: 
    chunks = make_chunks(audio, chunk_length)
    for i, chunk in enumerate(chunks):
      chunk_name = filename + "_chunk" + str(i)
      count += 1
      print(chunk_name)
      chunk.export(pathname + "/" + chunk_name, format = "wav") 

537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk1
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk2
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk3
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk4
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk5
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk6
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk7
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk8
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk9
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk10
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk11
537199__georgehopkins__amb

In [ ]:
# Check number of files generated
print(count)

334


In [ ]:
# remove files >4s since we have already splitted them into chunks of 4s each
# remove audio files <2s as we dont want to pad each audio file with too much silences
total_duration = 0
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  if audio.duration_seconds > 4 or audio.duration_seconds <= 2: 
    os.remove(pathname + "/" + filename)

In [ ]:
# Check duration of each audio track
filecount = 0
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  filecount += 1
  print(filename)
  print(audio.duration_seconds)

537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk0
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk1
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk2
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk3
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk4
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk5
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk6
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk7
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk8
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk9
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-water.wav_chunk10
4.0
537199__georgehopkins__ambience-english-summer-birdsong-running-

In [ ]:
# Check total number of files
print (filecount)

323


#### Pad audio data that are <4s in length with silences at the back

In [ ]:
# Check duration of audio data <4s
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  if audio.duration_seconds < 4: 
    print(filename)
    print(audio.duration_seconds)

364663__jmiddlesworth__bird-call-in-spring.wav_chunk32
3.5529931972789117
freesound_ambience_0012.wav_chunk17
2.209
own_ambience_0004.wav_chunk7
2.0229931972789115
soundsnap_ambience_0006.wav_chunk10
3.563
soundsnap_ambience_0021.wav_chunk9
2.7189795918367348


In [ ]:
# Pad each audio with silences until their duration hit 4s
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  if audio.duration_seconds < 4: 
    pad_ms = 4000 - (audio.duration_seconds * 1000) # milliseconds of silences needed
    silence = AudioSegment.silent(duration=pad_ms)
    padded = audio + silence
    print(filename)
    print(padded.duration_seconds)

364663__jmiddlesworth__bird-call-in-spring.wav_chunk32
3.999909297052154
freesound_ambience_0012.wav_chunk17
3.9998541666666667
own_ambience_0004.wav_chunk7
3.9998866213151927
soundsnap_ambience_0006.wav_chunk10
3.999833333333333
soundsnap_ambience_0021.wav_chunk9
3.999909297052154


In [ ]:
filecount = 0
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  filecount += 1

print(filecount)

323


In [ ]:
# Export and replace the files
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  if audio.duration_seconds < 4: 
    pad_ms = 4000 - (audio.duration_seconds * 1000) # milliseconds of silences needed
    silence = AudioSegment.silent(duration=pad_ms)
    padded = audio + silence
    padded.export(pathname + "/" + filename, format='wav')

In [ ]:
# Check total number of audio files
filecount = 0
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  filecount += 1

print(filecount)

323


In [ ]:
# Check if there are still audio files >4s
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  if audio.duration_seconds > 4: 
    print(filename)
    print(audio.duration_seconds)

In [ ]:
# Renaming every file within the dataset
category = "ambience"
count = 1;
for filename in os.listdir(pathname):
  os.rename(pathname + "/" + filename, pathname + "/" + category + " " + str(count) + ".wav")
  count += 1

In [ ]:
# Check duration of all audio files
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  print(filename)
  print(audio.duration_seconds)

ambience 1.wav
4.0
ambience 2.wav
4.0
ambience 3.wav
4.0
ambience 4.wav
4.0
ambience 5.wav
4.0
ambience 6.wav
4.0
ambience 7.wav
4.0
ambience 8.wav
4.0
ambience 9.wav
4.0
ambience 10.wav
4.0
ambience 11.wav
4.0
ambience 12.wav
4.0
ambience 13.wav
4.0
ambience 14.wav
4.0
ambience 15.wav
4.0
ambience 16.wav
4.0
ambience 17.wav
4.0
ambience 18.wav
4.0
ambience 19.wav
4.0
ambience 20.wav
4.0
ambience 21.wav
4.0
ambience 22.wav
4.0
ambience 23.wav
4.0
ambience 24.wav
4.0
ambience 25.wav
4.0
ambience 26.wav
4.0
ambience 27.wav
4.0
ambience 28.wav
4.0
ambience 29.wav
4.0
ambience 30.wav
4.0
ambience 31.wav
4.0
ambience 32.wav
4.0
ambience 33.wav
4.0
ambience 34.wav
4.0
ambience 35.wav
4.0
ambience 36.wav
4.0
ambience 37.wav
4.0
ambience 38.wav
4.0
ambience 39.wav
4.0
ambience 40.wav
4.0
ambience 41.wav
4.0
ambience 42.wav
4.0
ambience 43.wav
4.0
ambience 44.wav
4.0
ambience 45.wav
4.0
ambience 46.wav
4.0
ambience 47.wav
4.0
ambience 48.wav
4.0
ambience 49.wav
4.0
ambience 50.wav
4.0
ambience 

In [ ]:
# Check if any audio files not within the desired duration
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  if audio.duration_seconds != 4 and audio.duration_seconds < 3.999:
    print(filename)
    print(audio.duration_seconds)

In [ ]:
# Obtain total duration of audio tracks
total_sec = 0
for filename in os.listdir(pathname):
  audio = AudioSegment.from_file(pathname + "/" + filename)
  total_sec += audio.duration_seconds

print(total_sec)

1291.9993927154194
